In [1]:
from pyspark.sql import SparkSession
import getpass
username = getpass.getuser()
spark = SparkSession. \
builder. \
appName("Sneha Spark Session").\
config("spark.shuffle.useOldFetchProtocol", 'true'). \
config("spark.sql.warehouse.dir", f"/user/{username}/warehouse"). \
enableHiveSupport(). \
master('yarn'). \
getOrCreate()

In [2]:
spark

In [3]:
order_schema = 'order_id long, order_date date, customer_id long, order_status string'

In [4]:
order_df = spark.read.\
format("csv").\
schema(order_schema).\
load("/public/trendytech/orders/orders_1gb.csv")

In [5]:
customer_schema = "customer_id long,customer_fname string, customer_lname string,username string, password string,address string, city string, state string, pincode long"

In [6]:
customer_df = spark.read.format("csv").schema(customer_schema).load("/public/trendytech/retail_db/customers")

In [7]:
spark.conf.get('spark.sql.autoBroadcastJoinThreshold')

'10485760b'

In [8]:
order_df.join(customer_df,order_df.customer_id == customer_df.customer_id,"inner").write.format("noop").mode("overwrite").save()

In [9]:
spark.conf.set('spark.sql.autoBroadcastJoinThreshold','-1')

In [10]:
order_df.join(customer_df,order_df.customer_id == customer_df.customer_id,"inner").explain()

== Physical Plan ==
*(5) SortMergeJoin [customer_id#2L], [customer_id#8L], Inner
:- *(2) Sort [customer_id#2L ASC NULLS FIRST], false, 0
:  +- Exchange hashpartitioning(customer_id#2L, 200), ENSURE_REQUIREMENTS, [id=#67]
:     +- *(1) Filter isnotnull(customer_id#2L)
:        +- FileScan csv [order_id#0L,order_date#1,customer_id#2L,order_status#3] Batched: false, DataFilters: [isnotnull(customer_id#2L)], Format: CSV, Location: InMemoryFileIndex[hdfs://m01.itversity.com:9000/public/trendytech/orders/orders_1gb.csv], PartitionFilters: [], PushedFilters: [IsNotNull(customer_id)], ReadSchema: struct<order_id:bigint,order_date:date,customer_id:bigint,order_status:string>
+- *(4) Sort [customer_id#8L ASC NULLS FIRST], false, 0
   +- Exchange hashpartitioning(customer_id#8L, 200), ENSURE_REQUIREMENTS, [id=#75]
      +- *(3) Filter isnotnull(customer_id#8L)
         +- FileScan csv [customer_id#8L,customer_fname#9,customer_lname#10,username#11,password#12,address#13,city#14,state#15,pincode#16

In [11]:
order_df.join(customer_df,order_df.customer_id == customer_df.customer_id,"inner").write.format("noop").mode("overwrite").save()

In [12]:
spark.conf.set('spark.sql.autoBroadcastJoinThreshold','10485760b')

In [13]:
order_df.join(customer_df,order_df.customer_id == customer_df.customer_id,"inner").explain()

== Physical Plan ==
*(2) BroadcastHashJoin [customer_id#2L], [customer_id#8L], Inner, BuildRight, false
:- *(2) Filter isnotnull(customer_id#2L)
:  +- FileScan csv [order_id#0L,order_date#1,customer_id#2L,order_status#3] Batched: false, DataFilters: [isnotnull(customer_id#2L)], Format: CSV, Location: InMemoryFileIndex[hdfs://m01.itversity.com:9000/public/trendytech/orders/orders_1gb.csv], PartitionFilters: [], PushedFilters: [IsNotNull(customer_id)], ReadSchema: struct<order_id:bigint,order_date:date,customer_id:bigint,order_status:string>
+- BroadcastExchange HashedRelationBroadcastMode(List(input[0, bigint, false]),false), [id=#160]
   +- *(1) Filter isnotnull(customer_id#8L)
      +- FileScan csv [customer_id#8L,customer_fname#9,customer_lname#10,username#11,password#12,address#13,city#14,state#15,pincode#16L] Batched: false, DataFilters: [isnotnull(customer_id#8L)], Format: CSV, Location: InMemoryFileIndex[hdfs://m01.itversity.com:9000/public/trendytech/retail_db/customers], Partit

In [16]:
order_df.join(customer_df.hint("shuffle_hash"),order_df.customer_id == customer_df.customer_id,"inner").write.format("noop").mode("overwrite").save()

In [15]:
order_df.join(customer_df.hint("shuffle_merge"),order_df.customer_id == customer_df.customer_id,"inner").write.format("noop").mode("overwrite").save()